In [1]:
import os 
from openai import OpenAI

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
openai_api_key = os.getenv("OPENAI_API_KEY")

In [4]:
endpoint = "https://models.github.ai/inference"
model_name = "openai/gpt-4o"

client = OpenAI(
    base_url=endpoint,
    api_key=os.environ['GITHUB_API_TOKEN'],
    )

response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": "What is the capital of England?",
        }
    ],
    temperature=0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name

)



In [5]:
response.choices[0].message.content

'The capital of England is **London**.'

In [6]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

### Simple chain

In [10]:
model = "gpt-4o"

prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}."
)

model = ChatOpenAI(api_key=openai_api_key,temperature=0.0, model=model)
outputParser = StrOutputParser()

In [11]:
chain = prompt | model | outputParser

In [12]:
chain.invoke({"topic": "cats"})

"Sure! Here's a cat joke for you:\n\nWhy was the cat sitting on the computer?\n\nBecause it wanted to keep an eye on the mouse! 🐱🖱️"

### More complex Chains

In [13]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

In [15]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["Fares worked at Orenda", "chimps are mammals"],
    embedding=OpenAIEmbeddings(model="text-embedding-3-large",dimensions=1024)
)

retriever = vectorstore.as_retriever()

c:\Users\m.landoulsi\Desktop\pprojects\langchain\.venv\lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [16]:
retriever.get_relevant_documents("Where did Fares work?")

C:\Users\m.landoulsi\AppData\Local\Temp\ipykernel_26044\48198759.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents("Where did Fares work?")


[Document(metadata={}, page_content='Fares worked at Orenda'),
 Document(metadata={}, page_content='chimps are mammals')]

In [17]:
retriever.get_relevant_documents("Who are chimps?")

[Document(metadata={}, page_content='chimps are mammals'),
 Document(metadata={}, page_content='Fares worked at Orenda')]

In [18]:
template = """ Answer the question based on the context provided. 

Context: {context}

Question: {question} 
"""

prompt = ChatPromptTemplate.from_template(template)

In [19]:
from langchain.schema.runnable import RunnableMap

In [20]:
chain = RunnableMap({
    "context" : lambda x : retriever.get_relevant_documents(x["question"]),
    "question" : lambda x : x["question"]
}) | prompt | model | outputParser


In [21]:
chain.invoke({"question": "Where did Fares work?"})

'Fares worked at Orenda.'

In [22]:
inputs = RunnableMap({
    "context" : lambda x : retriever.get_relevant_documents(x["question"]),
    "question" : lambda x : x["question"]
})

In [24]:
inputs.invoke({"question": "Where did Fares work?"})

{'context': [Document(metadata={}, page_content='Fares worked at Orenda'),
  Document(metadata={}, page_content='chimps are mammals')],
 'question': 'Where did Fares work?'}

### Bind

In [25]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    }
  ]

In [28]:
model = "gpt-4o"

prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)

model = ChatOpenAI(api_key=openai_api_key,temperature=0.0, model=model).bind(functions=functions)


In [29]:
runnable = prompt | model

In [30]:
runnable.invoke({"input": "What is the weather at SFO?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'weather_search'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 64, 'total_tokens': 81, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'id': 'chatcmpl-BZZTNf9LbOYc6K6eeVrS0uuUVV3V3', 'service_tier': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run--9748853f-9912-44f5-9136-a0a8b95fa963-0', usage_metadata={'input_tokens': 64, 'output_tokens': 17, 'total_tokens': 81, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [31]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    },
        {
      "name": "sports_search",
      "description": "Search for news of recent sport events",
      "parameters": {
        "type": "object",
        "properties": {
          "team_name": {
            "type": "string",
            "description": "The sports team to search for"
          },
        },
        "required": ["team_name"]
      }
    }
  ]

In [32]:
model = model.bind(functions=functions)

In [33]:
runnable = prompt | model

In [34]:
runnable.invoke({"input": "How did Esperance de Tunis do in their last game?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"team_name":"Esperance de Tunis"}', 'name': 'sports_search'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 99, 'total_tokens': 119, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'id': 'chatcmpl-BZZV1bdhoCDmIPSPtS3uGlCVERvjf', 'service_tier': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run--5aa318d4-1476-47a8-b7bc-77c48ac8d4f5-0', usage_metadata={'input_tokens': 99, 'output_tokens': 20, 'total_tokens': 119, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### Fallbacks

In [ ]:
import json

simple_model = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0.0,
        openai_api_key=openai_api_key,
)
simple_chain = simple_model | (lambda x: json.loads(x.content))


In [44]:
challenge = "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"

In [45]:
simple_model.invoke(challenge)

AIMessage(content='Here is a JSON blob containing three poems, each represented as a JSON object with a title, author, and first line:\n\n```json\n{\n  "poems": [\n    {\n      "title": "Whispers of the Dawn",\n      "author": "Emily Carter",\n      "first_line": "In the hush of morning light, dreams take flight."\n    },\n    {\n      "title": "The Dance of Leaves",\n      "author": "Michael Thompson",\n      "first_line": "As autumn winds begin to weave, the trees start to sway."\n    },\n    {\n      "title": "Echoes of the Sea",\n      "author": "Sophia Lee",\n      "first_line": "Beneath the waves, a story sleeps, in whispers soft and deep."\n    }\n  ]\n}\n```', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 167, 'prompt_tokens': 31, 'total_tokens': 198, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0,

In [50]:
simple_chain.invoke(challenge)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [62]:
model = "o3-mini"
model = ChatOpenAI(
        model=model,
        openai_api_key=openai_api_key,
)

chain = model | (lambda x: json.loads(x.content))

In [63]:
model.invoke(challenge)

AIMessage(content='{\n  "poems": [\n    {\n      "title": "The Whispering Woods",\n      "author": "John Doe",\n      "first_line": "Beneath the ancient canopy, secrets whisper softly."\n    },\n    {\n      "title": "Sunrise Epiphany",\n      "author": "Jane Smith",\n      "first_line": "As dawn breaks, hope floods the horizon in golden hues."\n    },\n    {\n      "title": "Nocturnal Lullaby",\n      "author": "Alex Rivers",\n      "first_line": "In the quiet dark, the stars murmur forgotten melodies."\n    }\n  ]\n}', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 847, 'prompt_tokens': 30, 'total_tokens': 877, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 704, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'o3-mini-2025-01-31', 'system_fingerprint': 'fp_ded0d14823', 'id': 'chatcmpl-BZZq272ePgz50pM8gLcqzul8HYVog', 's

In [64]:
chain.invoke(challenge)

{'poems': [{'title': "The Dawn's Whisper",
   'author': 'Melody Rivers',
   'first_line': 'In the quiet blush of dawn, secrets softly unfurl.'},
  {'title': 'Midnight Echo',
   'author': 'Nightshade Monroe',
   'first_line': 'The moon murmurs its tales in the silent embrace of night.'},
  {'title': "Autumn's Lament",
   'author': 'Fiona Swift',
   'first_line': 'Falling leaves echo the sorrows of a bygone summer.'}]}

In [65]:
final_chain = simple_chain.with_fallbacks([chain])

In [66]:
final_chain.invoke(challenge)

{'poems': [{'title': 'The Silent River',
   'author': 'Aiden Brooks',
   'first_line': 'In the quiet of midnight, the river whispers ancient secrets.'},
  {'title': "Dawn's Tender Light",
   'author': 'Isabella Reed',
   'first_line': 'At first blush, the sky unfurls a soft embrace of golden hope.'},
  {'title': 'Echoes of the Past',
   'author': 'Evelyn Cruz',
   'first_line': 'Beneath crumbling arches, time murmurs stories of lost dreams.'}]}